In [12]:
from fuzzywuzzy import fuzz
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.sql.functions import udf
from Levenshtein import distance, ratio, jaro, jaro_winkler
from fuzzywuzzy import fuzz
from pyspark.sql.functions import udf

#### Levenshtein distance - minimum number of single character edits
#### Levenshtein ratio - similarity of two strings
#### Jaro - similarity score
#### Jaro-winkler - similarity score, which favors strings that match prefix from the beginning
#### fuzz partial ratio - gives a score based on how well parts of a string match another

In [21]:
def fuzz_partial_ratio(concept1, concept2):
    return fuzz.partial_ratio(concept1, concept2) / 100
distance_udf = udf(lambda x, y: distance(x,y), IntegerType())
ratio_udf = udf(lambda x, y: ratio(x,y), DoubleType())
jaro_udf = udf(lambda x, y: jaro(x,y), DoubleType())
jaro_winkler_udf = udf(lambda x, y: jaro_winkler(x,y), DoubleType())
fuzz_partial_ratio_udf = udf(lambda x, y: fuzz_partial_ratio(x,y), DoubleType())

In [16]:
phemlsample = spark.read \
    .option("header", "true") \
    .option("delimiter", ",") \
    .csv("file:/Users/xinzhuojiang/phenotype-features/pheMLSamplerefset.csv")

In [29]:
df = phemlsample.withColumn('distance', distance_udf('CONCEPT_ID_1', 'CONCEPT_ID_2')) \
.withColumn('ratio', ratio_udf('CONCEPT_ID_1', 'CONCEPT_ID_2')) \
.withColumn('jaro', jaro_udf('CONCEPT_ID_1', 'CONCEPT_ID_2')) \
.withColumn('jaro_wrinkler_udf', jaro_winkler_udf('CONCEPT_ID_1', 'CONCEPT_ID_2')) \
.withColumn('fuzz_partial_ratio', fuzz_partial_ratio_udf('CONCEPT_ID_1', 'CONCEPT_ID_2'))

In [30]:
df.repartition(1).write.option('header', 'true').csv('lexical_udf')